In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
demand = pd.read_csv('../input/schedule_5min_0612.csv')
demand['schedule'] = demand['schedule'] // 5 + 1

In [9]:
cs = np.array([0.0129,0.0133,0.0137,0.0142,0.0147,
                                0.0153,0.0158,0.0166,0.0172,0.018,
                                0.0188,0.0197,0.0207,0.0219,0.0231,
                                0.0245,0.026,0.0278,0.03,0.0323,
                                0.0351,0.0384,0.0423,0.0472,0.0536,
                                0.0617,0.0726,0.0887,0.1136,0.1582,
                                0.2622,0.9278,])*60
cs = np.cumsum(cs)
soc_transition_time




array([ 0.774,  0.798,  0.822,  0.852,  0.882,  0.918,  0.948,  0.996,
        1.032,  1.08 ,  1.128,  1.182,  1.242,  1.314,  1.386,  1.47 ,
        1.56 ,  1.668,  1.8  ,  1.938,  2.106,  2.304,  2.538,  2.832,
        3.216,  3.702,  4.356,  5.322,  6.816,  9.492, 15.732, 55.668])

In [10]:
cs

array([  0.774,   1.572,   2.394,   3.246,   4.128,   5.046,   5.994,
         6.99 ,   8.022,   9.102,  10.23 ,  11.412,  12.654,  13.968,
        15.354,  16.824,  18.384,  20.052,  21.852,  23.79 ,  25.896,
        28.2  ,  30.738,  33.57 ,  36.786,  40.488,  44.844,  50.166,
        56.982,  66.474,  82.206, 137.874])